In [66]:
import pandas as pd

def stochastic(df: pd.DataFrame, num_days: int) -> pd.DataFrame:
    min_low = df['Low'].rolling(num_days).min()
    max_high = df['High'].rolling(num_days).max()
    return ((df['Close']-min_low)/(max_high-min_low)).rolling(3).mean().rolling(3).mean()

def support_levels(df: pd.DataFrame):
    pivot = df[['High', 'Low', 'Close']].shift(1).mean(axis=1)
    return pivot * 2 - df['High'].shift(1)

df = pd.read_csv('data/spy_price.csv')
df = df.join(pd.read_csv('data/spy_vs_25dr.csv').drop('Date', axis=1), )
df['Support'] = support_levels(df)
df['Stochastic'] = stochastic(df, 14)
print(df.tail())


           Date        Open        High         Low       Close   Adj Close  \
751  2024-03-15  510.209991  511.700012  508.119995  509.829987  509.829987   
752  2024-03-18  514.000000  515.479980  512.440002  512.859985  512.859985   
753  2024-03-19  512.150024  516.000000  511.119995  515.710022  515.710022   
754  2024-03-20  515.770020  520.619995  515.080017  520.479980  520.479980   
755  2024-03-21  523.390015  524.109985  521.914978  522.200012  522.200012   

        Volume  25-Delta Put IV  25-Delta Call IV  Put - Call Diff  \
751  107585800             13.7              11.6              2.1   
752   88893300             13.3              11.3              2.0   
753   60755300             12.7              11.1              1.6   
754   69594600             11.9              10.6              1.3   
755   59463663             11.8              10.5              1.3   

        Support  Stochastic  
751  512.136678    0.768845  
752  508.066651    0.688275  
753  511.70666

In [69]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1, subplot_titles=['SPY Chart', 'SPY P-C IV Skew'], shared_xaxes=True)

fig.add_candlestick(
    x = df['Date'],
    open = df['Open'],
    high = df['High'],
    low = df['Low'],
    close = df['Close'],
    name='SPY',
    row=1,
    col=1
)
fig.update_xaxes(rangeslider_visible=False)
fig.add_scatter(x=df['Date'], y=df['Support'], name='Support', row=1, col=1)
fig.add_scatter(x=df['Date'], y=df['Put - Call Diff'], name='IV Skew', row=2, col=1)
fig.add_scatter(x=df['Date'], y=df['Stochastic'], name='Stochastic', row=3, col=1)
fig.add_scatter(x=df['Date'], y=df['Low']-df['Support'], name='Support', row=4, col=1)
fig.add_hline(y=0.2, row=3, col=1)
fig.add_hline(y=0.8, row=3, col=1)

fig.show()